In [1]:
import extcolors
import pandas as pd
from colormap import rgb2hex
from matplotlib import pyplot as plt
from PIL import Image

In [2]:
import sys, os

sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(os.path.abspath(os.path.join("../scripts")))

In [3]:
import color_extraction 

extract_color = color_extraction.extract_color()

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

path_to_dataset = os.getenv('path_to_provided_data')
path_to_extracted = os.getenv('path_to_extracted_data')

In [6]:
# load the performance data, we only need the game_id data

data = pd.read_csv(path_to_dataset+'performance_data.csv')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   game_id       905 non-null    object 
 1   preview_link  905 non-null    object 
 2   ER            905 non-null    float64
 3   CTR           905 non-null    float64
dtypes: float64(2), object(2)
memory usage: 28.4+ KB


In [8]:
# example of 1 pic
gameid = data.loc[0, 'game_id']
path_to_pic = path_to_extracted + gameid + '/' + 'start_frame' + '.png'

In [12]:
# colorExt.identify_color_composition(path_to_pic, tolerance=12, limit=10, visualize=True)
extract_color.identify_color_composition(path_to_pic, tolerance=12, limit=10, visualize=False)

,c_code,rgb,occurrence
0,#FFFFFF,"(255, 255, 255)",404789
1,#D3E6E5,"(211, 230, 229)",48331
2,#434343,"(67, 67, 67)",34293
3,#7F7F7F,"(127, 127, 127)",22028
4,#C3C6BF,"(195, 198, 191)",21865
5,#7F6154,"(127, 97, 84)",18726
6,#000000,"(0, 0, 0)",15107
7,#5B4737,"(91, 71, 55)",11635
8,#232323,"(35, 35, 35)",11126
9,#A27A66,"(162, 122, 102)",9447


In [11]:
# extract 4 dominant colors

extract_color.identify_dominant_colors(path_to_pic, tolerance=12, top=4)

['#FFFFFF',
 '#D3E6E5',
 '#434343',
 '#7F7F7F',
 0.632542113323124,
 0.07552426790011564,
 0.05358783635965872,
 0.034421977060349405]

In [17]:
# try a mini df

gameids = data.loc[:10, 'game_id']

df = pd.DataFrame(columns=['game_id', 'dom_color_1', 'dom_color_2', 'dom_color_3', 'dom_prct_1', 'dom_prct_2', 'dom_prct_3'])


for i, gameid in enumerate(gameids):
    path_to_pic = path_to_extracted + gameid + '/' + 'start_frame' + '.png'
    dom_colors = extract_color.identify_dominant_colors(path_to_pic, tolerance=12, top=3)
    row = [gameid] + dom_colors
    df.loc[i] = row

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 10
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   game_id      11 non-null     object 
 1   dom_color_1  11 non-null     object 
 2   dom_color_2  11 non-null     object 
 3   dom_color_3  11 non-null     object 
 4   dom_prct_1   11 non-null     float64
 5   dom_prct_2   11 non-null     float64
 6   dom_prct_3   11 non-null     float64
dtypes: float64(3), object(4)
memory usage: 704.0+ bytes
